In [24]:
### create NETCDF data files to publish with paper

In [ ]:
import numpy as np
import datetime
import tqdm
import os
import netCDF4 as nc4
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

In [25]:
# create dates list
days = np.arange(0,4018)
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=np.int(days)) for days in days]

# load Will's lons & lats, 160 x 160 array
lats = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lat.npy')
lons = np.load('/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_processed/Robbie_lon.npy')

# import freeboard data
sats = ['CS2_CPOM','CS2_LARM']
datapath = '/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_interpolated/'
folders = ['CS2_CPOM/','Bristol_LARM/CS2/freeboard/']

CS2_CPOM_fb = np.full((len(days),160,160), np.nan)
CS2_LARM_fb = np.full((len(days),160,160), np.nan)

dics = [CS2_CPOM_fb, CS2_LARM_fb]

for sat, folder, dic in zip (sats,folders, dics):
    location = datapath + folder
    for day in tqdm.tqdm(days):
        date = dates[day].strftime('%Y%m%d')
        for f in os.listdir(location):
            if 'FB_interp' in f:
                if date in f:
                    dic[day] = np.load(location+str(f)) 
                else: 
                    pass

# import uncertainty data
CS2_CPOM_unc = np.full((len(days),160,160), np.nan)
CS2_LARM_unc = np.full((len(days),160,160), np.nan)

dics = [CS2_CPOM_unc, CS2_LARM_unc]

for sat, folder, dic in zip (sats,folders, dics):
    location = datapath + folder
    for day in tqdm.tqdm(days):
        date = dates[day].strftime('%Y%m%d')
        for f in os.listdir(location):
            if 'FB_uncertainty' in f:
                if date in f:
                    dic[day] = np.load(location+str(f)) 
                else: 
                    pass

100%|██████████████████████████████████████| 4018/4018 [00:15<00:00, 261.38it/s]


In [26]:
# create CS2_CPOM NETCDF
data_vars = {'Radar Freeboard':(['t','x','y'],CS2_CPOM_fb),
            'Uncertainty':(['t','x','y'],CS2_CPOM_unc)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'lon':(['x','y'],lons),
                        'lat':(['x','y'],lats),
                         'day':(['t'],days)}    )
ds.to_netcdf(f'/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_interpolated/CS2_CPOM/CS2_CPOM.nc','w')

In [32]:
# create CS2_LARM NETCDF
data_vars = {'Radar Freeboard':(['t','x','y'],CS2_LARM_fb),
            'Uncertainty':(['t','x','y'],CS2_LARM_unc)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'lon':(['x','y'],lons),
                        'lat':(['x','y'],lats),
                         'date':(['t'],days)}    )
ds.to_netcdf(f'/Users/carmennab/PhD/OI_PolarSnow/freeboard_daily_interpolated/Bristol_LARM/CS2/freeboard/CS2_LARM.nc','w')